***Attention this dataloader requires almost 400GB of free harddisk space!!!!***

***And takes on a powerfull machine around 4 hours to run!***

In [72]:
import os
from os import path
import numpy as np
import db_builder.db_handler as dbh
from getpass import getpass
from datetime import datetime
import datetime
from torchvision import transforms
import cv2
from tqdm import tqdm

In [73]:
db_params = {
    'user': 'postgres',
    'password': getpass('Please enter DB pw'),  # enter your DB password
    'host': 'localhost',  # 'localhost' or IP address
    'port': '5432',  # '5432'
    'database': 'ttdatabase',  #tensionTerminator
}
toolcheck = dbh.DB_Conn(db_params)
toolcheck.connect()
engine = toolcheck.get_engine()

Connected to PostgreSQL, DB: ttdatabase


In [74]:
data_df = toolcheck.get_combined_labeled_data()
toolcheck.disconnect()

2023-11-29 07:24:46,204 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-11-29 07:24:46,205 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-29 07:24:46,206 INFO sqlalchemy.engine.Engine select current_schema()
2023-11-29 07:24:46,207 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-29 07:24:46,208 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-11-29 07:24:46,208 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-29 07:24:46,209 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-29 07:24:46,209 INFO sqlalchemy.engine.Engine 
            SELECT l.id AS loop_id, v.file_path, u.name, u.surename
            FROM loop l
            LEFT JOIN video v ON l.id = v.loop_id AND v.device = 'rgbCam'
            LEFT JOIN users u ON l.user_id = u.id
            WHERE l.human_labeled = true
        
2023-11-29 07:24:46,210 INFO sqlalchemy.engine.Engine [generated in 0.00076s] {}
2023-11-29 07:24:46,215 INFO sqlalchemy.engine.Engine ROLLBACK
Connection to tt

In [75]:
data_df

,loop_id,file_path,name,surename
0,1,D:/BigBlock_01/tt_video_data/videos-2023-09-08...,Martin,Hofer
1,2,D:/BigBlock_01/tt_video_data/videos-2023-09-08...,Juergen,Zangerl
2,3,D:/BigBlock_01/tt_video_data/videos-2023-09-08...,Christina,Greiderer
3,4,D:/BigBlock_01/tt_video_data/videos-2023-09-08...,Philipp,Egger
4,5,D:/BigBlock_01/tt_video_data/videos-2023-09-08...,Lukas,Prenner
...,...,...,...,...
123,124,D:/BigBlock_01/tt_video_data/videos-2023-09-18...,Juergen,Zangerl
124,125,D:/BigBlock_01/tt_video_data/videos-2023-09-18...,Juergen,Zangerl
125,126,D:/BigBlock_01/tt_video_data/videos-2023-09-18...,Juergen,Zangerl
126,127,D:/BigBlock_01/tt_video_data/videos-2023-09-18...,Juergen,Zangerl


In [76]:
def trigger_crop(image):
    crop_box = (400, 450, 550, 550)
    cropped_image = transforms.functional.crop(image, *crop_box)
    return cropped_image

In [77]:
def get_time():
    return datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")

In [78]:
def video_to_image_converter(source_path: str, output_path: str, crop=False):
    vidcap = cv2.VideoCapture(source_path)
    os.makedirs(output_path, exist_ok=True)
    count = 0

    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = 1

    while count < total_frames:
        success, image = vidcap.read()

        if count % int(fps) == 0:
            if not image is None and not image.size == 0:
                transform_test = transforms.ToPILImage()
                image = transform_test(image)
                if crop:
                    image = trigger_crop(image)
                image = np.asarray(image)
                cv2.imwrite(f"{output_path}/{get_time()}_{count}.png", image)  # save frame as PNG file

        count += 1

    vidcap.release()
    cv2.destroyAllWindows()

In [79]:
def save_path(label: str, time):
    abs_path = os.getcwd()
    two_up =  path.abspath(path.join(abs_path ,"../.."))
    return path.join(two_up, 'data', f'{os.path.basename(abs_path)}', f'{time}', label)

In [80]:
current_time = get_time()

for index, row in tqdm(data_df.iterrows()):
    
    video_to_image_converter(row['file_path'],save_path(f"{row['name']}_{row['surename']}",current_time))
    # Accessing row data
    #print(f"Index: {index}")
    #print(f"Loop ID: {row['loop_id']}, Name: {row['name']}, Surname: {row['surename']}")


In [81]:
for index in tqdm(range(len(data_df))):
    row = data_df.iloc[index] 
    video_to_image_converter(row['file_path'], save_path(f"{row['name']}_{row['surename']}", current_time))


100%|██████████| 128/128 [4:03:30<00:00, 114.14s/it] 
